<a href="https://colab.research.google.com/github/talktovishal/NLP-ML-Projects/blob/master/Project2_word2vec_Part1_DownloadPreprocessKaggleData_Frankenstein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "/content/drive/My Drive/Public Shared/Project-2-Word-Embeddings/frankenstein.txt" "frankenstein.txt"


In [0]:
!pwd

/content


In [0]:
!ls -a -l

total 460
drwxr-xr-x 1 root root   4096 Apr  5 23:45 .
drwxr-xr-x 1 root root   4096 Apr  5 23:22 ..
drwxr-xr-x 1 root root   4096 Apr  2 16:11 .config
drwx------ 4 root root   4096 Apr  5 23:43 drive
-rw------- 1 root root 447984 Apr  5 23:45 frankenstein.txt
drwxr-xr-x 1 root root   4096 Mar 18 16:23 sample_data


In [0]:
import nltk
#for stop words
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import string

lemmatizer = WordNetLemmatizer()

#lemmatizer code from https://simonhessner.de/lemmatize-whole-sentences-with-python-and-nltks-wordnetlemmatizer/
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:          
    return None

def lemmatize_sentence(sentence):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:            
      res_words.append(word)
    else:
      res_words.append(lemmatizer.lemmatize(word, tag))
  return res_words


import re
# If you need to use the regex more than once it is suggested to compile it.
notStartsWithAlphaNumber = re.compile(r"[^\w\s]")
onlyDigits = re.compile(r"\d+")

def normalizeSentence(sentence):
  #lowercase
  #There's a risk of doing lower(). US and us are two different things
  updatedSentence = sentence.lower()
  #remove punctuations
  updatedSentence = updatedSentence.translate(str.maketrans('','',string.punctuation))
  updatedSentence = notStartsWithAlphaNumber.sub("", updatedSentence)
  updatedSentence = onlyDigits.sub("", updatedSentence)  
  return updatedSentence


#not i am not removing stop words since i plan to use this for tf-idf calculation
def normalizeAndLemmatizeSentence(sentence):
  updatedSentence = normalizeSentence(sentence)
  #lemmatization
  return lemmatize_sentence(updatedSentence)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
import nltk.data
nltk.download('punkt')
sentence_detection = nltk.data.load('tokenizers/punkt/english.pickle')

#https://www.nltk.org/api/nltk.tokenize.html
def extract_sentences(text):
  return sentence_detection.tokenize(text)


def extract_n_normalize_sentences(text):
  allSentences = sentence_detection.tokenize(text)
  updatedSentences = []
  for sentence in allSentences:
    #NOTE: You need an additional [] bracket before you add content to the list
    #Reason = https://stackoverflow.com/questions/29947007/array-extendstring-adds-every-character-instead-of-just-the-string
    updatedSentences.extend([normalizeSentence(sentence)])
  return updatedSentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
temp = open('frankenstein.txt','r').read().split('\n')
seaparator = ' '
content = seaparator.join(temp)

normalizedSentences = []
normalizedSentences.extend([normalizeSentence(s) for s in extract_sentences(content)])

"""
for file in os.listdir():
    if(file.endswith('.txt')):
      pdf = pd.read_csv(file)
      print(f'Read file {file} into a panda dataframe')
      
      #this is a very slow method to iterate over the list and apply the normlize function.
      #do somethign better: https://chrisalbon.com/python/basics/applying_functions_to_list_items/
      #pdf['content'].apply(lambda content: normalizedSentences.extend(extract_n_normalize_sentences(content)))

      pdf['content'].apply(lambda content: normalizedSentences.extend([normalizeSentence(s) for s in extract_sentences(content)]))
      #for testing only, read only one file...
      #break
      #pdf.info()
      #pdf.head(3)
"""

"\nfor file in os.listdir():\n    if(file.endswith('.txt')):\n      pdf = pd.read_csv(file)\n      print(f'Read file {file} into a panda dataframe')\n      \n      #this is a very slow method to iterate over the list and apply the normlize function.\n      #do somethign better: https://chrisalbon.com/python/basics/applying_functions_to_list_items/\n      #pdf['content'].apply(lambda content: normalizedSentences.extend(extract_n_normalize_sentences(content)))\n\n      pdf['content'].apply(lambda content: normalizedSentences.extend([normalizeSentence(s) for s in extract_sentences(content)]))\n      #for testing only, read only one file...\n      #break\n      #pdf.info()\n      #pdf.head(3)\n"

In [0]:
normalizedSentences[555]

'no one can conceive the variety of feelings which bore me onwards like a hurricane in the first enthusiasm of success'

In [0]:
import pickle
with open('normalizedSentences-frankenstein.pickle', 'wb') as f:
    pickle.dump(normalizedSentences, f)

!cp "normalizedSentences-frankenstein.pickle" "/content/drive/My Drive/Public Shared/Project-2-Word-Embeddings/normalizedSentences-frankenstein.pickle"

